In [1]:
import pandas as pd
import numpy as np
import math
import os
from test_class import Test
os.environ['PYTHONHASHSEED'] = str(50)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt

In [2]:
rounds = 50
dataset = pd.read_csv('RTT_data.csv')
data_args = {
    'data seed': 11,
    'distance clients': [0, 2, 3, 6],
    'distance augments': [0.5, 0.5, 0.5, 0.5],
    'tolerance': 5,
    'exclude dtypes': 'object',
    'drop labels': ['GroundTruthRange[m]'],
    'target labels': ['GroundTruthRange[m]'],
    'test size': 0.2,
    'normalize': True, 
    'client num': None
}

Mt = (np.ones(rounds) * 5).astype('int32')
model_seed = 50
test = Test(dataset, data_args, Mt, model_seed)

In [3]:
test.split(scheme = 3, args = (3, 4))
test.display_metadata()

Number of samples: 29581
Features per sample: 10

Columns:
%Timestamp[s], GroundTruthPositionX[m], GroundTruthPositionY[m], GroundTruthPositionZ[m], ML_range_estimate[m], AP_positionX[m], AP_positionY[m], AP_positionZ[m], AP_index, ToD_factor[m]

Clients: 12
Total Client Training Samples: 23659 (79.98%)
Total Client Training Labels: 23659
Total Test Samples: 5922 (20.02%)
Total Test Labels: 5922


In [4]:
test.display_client_distribution()

Data Distribution
Client 1: 2.75%
Client 2: 2.42%
Client 3: 6.02%
Client 4: 14.65%
Client 5: 1.14%
Client 6: 1.97%
Client 7: 0.04%
Client 8: 15.32%
Client 9: 12.44%
Client 10: 18.54%
Client 11: 14.83%
Client 12: 9.88%

Distance Distribution w.r.t. Max Distance
Client 1: 68.42%
Client 2: 1.13%
Client 3: 84.83%
Client 4: 100.00%
Client 5: 24.36%
Client 6: 28.14%
Client 7: 58.71%
Client 8: 28.25%
Client 9: 54.60%
Client 10: 49.32%
Client 11: 42.32%
Client 12: 6.30%


In [5]:
# perform cross validation over different seeds
seeds = [1, 2, 3, 4, 5]
fedavg_scores = []
conf_scores = []
for s in seeds:
    test.data_args['data seed'] = s
    cv_data = test.cross_validation(K = 7, score = 1, a = 1, l = 1000, args = (3, 4), context = [0, 2])
    s1 = cv_data['s1 score']
    fedavg_scores.append(s1[0])
    conf_scores.append(s1[1])

CV Pair 1
FedAvg
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 111.77s

ConFeddi
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 125.04s

FedAvg Avg Test Error: 0.2419380212461042
ConFeddi Avg Test Error: 0.24266468547251233
FedAvg MSE: 0.11963135004043579
ConFeddi MSE: 0.11286627501249313

CV Pair 2
FedAvg
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 113.49s

ConFeddi
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 124.9s

FedAvg Avg Test Error: 0.22957650003085167
ConFeddi Avg Test Error: 0.2507616653474391
FedAvg MSE: 0.06982829421758652
ConFeddi MSE: 0.08325766772031784

CV Pair 3
FedAvg
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 112.3s

ConFeddi
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 1

In [6]:
print(f'FedAvg Average S1 Score: {np.array(fedavg_scores).mean():.5f}')

FedAvg Average S1 Score: 0.23066


In [7]:
print(f'ConFeddi Average S1 Score: {np.array(conf_scores).mean():.5f}')

ConFeddi Average S1 Score: 0.22672


In [8]:
print(f'FedAvg S1 Score Variance: {np.array(fedavg_scores).var():.5f}')

FedAvg S1 Score Variance: 0.00138


In [9]:
print(f'ConFeddi S1 Score Variance: {np.array(conf_scores).var():.5f}')

ConFeddi S1 Score Variance: 0.00133
